In [2]:
import pandas as pd

# Read the csv file
df = pd.read_csv('D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\MainDataset.csv')

# Print it out if you want
print(df.shape, "\n")
print("This dataset consists of following features:\n", df.columns) # getting the features' name

(14853, 15) 

This dataset consists of following features:
 Index(['id', 'reviewer_id', 'sample_id', 'smell', 'severity',
       'review_timestamp', 'type', 'code_name', 'repository', 'commit_hash',
       'path', 'start_line', 'end_line', 'link',
       'is_from_industry_relevant_project'],
      dtype='object')


In the next we're going to delete the rows containing none value for severity. These rows are useless in our computations.

In [3]:
dfWithoutNone = df[df['severity'] != "none"]
dfWithoutNone.shape
dfWithoutNone.to_csv("D:\\Articles\\Code Smells\\CodeSmells\\Smell Project\\DatasetWithoutNone.csv")

In [28]:
import requests
import lxml
from bs4 import BeautifulSoup

url = "https://github.com/apache/tomcat/blob/a9c1a0661198d9ba37c1facd8385fe05d538c4ad/java/org/apache/el/parser/ELParser.java/#L140-L168"
headers = {
  'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/63.0.3239.132 Safari/537.36 QIHU 360SE'
}
f = requests.get(url, headers = headers)

soup = BeautifulSoup(f.content,'lxml')
print(soup)

movies = soup.find('td', {'id':'L140'})
print(movies)

<!DOCTYPE html>
<html data-a11y-animated-images="system" data-color-mode="auto" data-dark-theme="dark" data-light-theme="light" lang="en">
<head>
<meta charset="utf-8"/>
<link href="https://github.githubassets.com" rel="dns-prefetch"/>
<link href="https://avatars.githubusercontent.com" rel="dns-prefetch"/>
<link href="https://github-cloud.s3.amazonaws.com" rel="dns-prefetch"/>
<link href="https://user-images.githubusercontent.com/" rel="dns-prefetch"/>
<link crossorigin="" href="https://github.githubassets.com" rel="preconnect"/>
<link href="https://avatars.githubusercontent.com" rel="preconnect"/>
<link crossorigin="anonymous" href="https://github.githubassets.com/assets/light-5178aee0ee76.css" integrity="sha512-UXiu4O52iBFkqt6Kx5t+pqHYP2/LWWIw9+l5ia74TWw+xPzpH44BFfAQp7yzCe0XFGZa72Xiqyml6tox1KkUjw==" media="all" rel="stylesheet"/><link crossorigin="anonymous" href="https://github.githubassets.com/assets/dark-217d4f9c8e70.css" integrity="sha512-IX1PnI5wWBz8Kgb1JI0f2QFa/WuRQQHJHe0vkKinQ